# Optimization with Scipy Optimize

____________
## Unconstrained optimization

main source: https://ipython-books.github.io/

In [ ]:
import numpy as np
import scipy as sp
import scipy.optimize as opt
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
plt.style.use('ggplot')
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 5)

$$\min_x \ 1-\frac{sin(x)}{x}$$

In [ ]:
def f(x):
    return 1 - np.sin(x) / x

In [ ]:
x = np.linspace(-20., 20., 1000)
y = f(x)
fig, ax = plt.subplots(1, 1)
ax.plot(x, y)

In [ ]:
x0 = 3
xmin = opt.minimize(f, x0).x
fig, ax = plt.subplots(1, 1)
ax.plot(x, y)
ax.scatter(x0, f(x0), marker='o', s=300)
ax.scatter(xmin, f(xmin), marker='v', s=300,
           zorder=20)
ax.set_xlim(-20, 20)

In [ ]:
x0 = 10
xmin = opt.minimize(f, x0).x
fig, ax = plt.subplots(1, 1)
ax.plot(x, y)
ax.scatter(x0, f(x0), marker='o', s=300)
ax.scatter(xmin, f(xmin), marker='v', s=300,
           zorder=20)
ax.set_xlim(-20, 20)

In [ ]:
x0=20
xmin = opt.basinhopping(f, x0, 1000).x
fig, ax = plt.subplots(1, 1)
ax.plot(x, y)
ax.scatter(x0, f(x0), marker='o', s=300)
ax.scatter(xmin, f(xmin), marker='v', s=300,
           zorder=20)
ax.set_xlim(-20, 20)

#### Lévi function:

$$\min_{x,y} \ f(x,y) = \sin^{2}\left(3\pi x\right)+\left(x-1\right)^{2}\left(1+\sin^{2}\left(3\pi y\right)\right)+\left(y-1\right)^{2}\left(1+\sin^{2}\left(2\pi y\right)\right)$$

In [ ]:
def g(X):
    # X is a 2*N matrix, each column contains
    # x and y coordinates.
    x, y = X
    return (np.sin(3*np.pi*x)**2 +
            (x-1)**2 * (1 + np.sin(3*np.pi*y)**2) +
            (y-1)**2 * (1 + np.sin(2*np.pi*y)**2))

In [ ]:
n = 500
k = 10
X, Y = np.mgrid[-k:k:n * 1j,
                -k:k:n * 1j]
Z = g(np.vstack((X.ravel(), Y.ravel()))).reshape(n, n)
fig, ax = plt.subplots(1, 1)
# We use a logarithmic scale for the color here.
ax.imshow(np.log(Z), cmap=plt.cm.hot_r,
          extent=(-k, k, -k, k), origin='upper')
ax.set_axis_off()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
# Plot the surface
ax.plot_surface(X, Y, Z, color='b')

plt.show()

In [ ]:
# We use the Powell method.
x0=10
y0=10
xsol, ysol = opt.minimize(g, (x0, y0),method='Powell').x
FO = opt.minimize(g, (x0, y0),method='Powell').fun
fig, ax = plt.subplots(1, 1)
ax.imshow(np.log(Z), cmap=plt.cm.hot_r,
          extent=(-k, k, -k, k), origin='upper')
ax.scatter(xsol, ysol, s=100)
ax.set_axis_off()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
# Plot the surface
ax.plot_surface(X, Y, Z, color='b')
ax.scatter(x0, y0, FO, s=100)
plt.show()

### Constrained Optimization
source: https://people.duke.edu/~ccc14/sta-663/BlackBoxOptimization.html

$$\min_{x,y} f(x,y) = -(2xy + 2x - x^2 -2y^2)$$
$$s.t.\quad x^3 - y = 0$$
$$ y - (x-1)^4 - 2\geq 0$$
$$0.5 \le x \le 1.5$$
$$1.5 \le y \le 2.5$$

In [ ]:
def f(x):
    return -(2*x[0]*x[1] + 2*x[0] - x[0]**2 - 2*x[1]**2)

fig = plt.figure()
x = np.linspace(0, 3, 100)
y = np.linspace(0, 3, 100)
X, Y = np.meshgrid(x, y)
Z = f(np.vstack([X.ravel(), Y.ravel()])).reshape((100,100))
plt.contour(X, Y, Z, np.arange(-1.99,10, 1));
plt.plot(x, x**3, 'k:', linewidth=1)
plt.plot(x, (x-1)**4+2, 'k:', linewidth=1)
plt.fill([0.5,0.5,1.5,1.5], [2.5,1.5,1.5,2.5], alpha=0.3)
plt.axis([0,3,0,3])
plt.show()

In [ ]:
cons = ({'type': 'eq',
         'fun' : lambda x: np.array([x[0]**3 - x[1]]),
         'jac' : lambda x: np.array([3.0*(x[0]**2.0), -1.0])},
        {'type': 'ineq',
         'fun' : lambda x: np.array([x[1] - (x[0]-1)**4 - 2])})

bnds = ((0.5, 1.5), (1.5, 2.5))
x0 = [0, 2.5]

#uncostrained solution
ux = opt.minimize(f, x0, constraints=None)
ux

In [ ]:
#constrained solution
cx = opt.minimize(f, x0, bounds=bnds,constraints=cons)
cx

In [ ]:
x = np.linspace(0, 3, 100)
y = np.linspace(0, 3, 100)
X, Y = np.meshgrid(x, y)
Z = f(np.vstack([X.ravel(), Y.ravel()])).reshape((100,100))
fig = plt.figure()
plt.contour(X, Y, Z, np.arange(-1.99,10, 1));
plt.plot(x, x**3, 'k:', linewidth=1)
plt.plot(x, (x-1)**4+2, 'k:', linewidth=1)
plt.text(ux['x'][0], ux['x'][1], 'x', va='center', ha='center', size=20, color='blue')
plt.text(cx['x'][0], cx['x'][1], 'x', va='center', ha='center', size=20, color='red')
plt.fill([0.5,0.5,1.5,1.5], [2.5,1.5,1.5,2.5], alpha=0.3)
plt.axis([0,3,0,3]);